In [2]:
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import control as cn
import numpy as np
import math

In [ ]:
def eulersApproximation(t, y, ls, epoch):
    for i in range(epoch):
        
        y = 
        ls.append()